In [ ]:
!pip install requests

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import requests

# API 호출 URL과 인증키
url = 'http://apis.data.go.kr/1160100/service/GetFinaStatInfoService_V2/getSummFinaStat_V2'
service_key = '3phTHgOnMKftn3MRUX9XsCGgzB5%2FVlozxJc16E68cE6ANiIoXsL9dprcwxB3c3KUfMJqaY4oPQwE8HraWBJYGg%3D%3D'

# 요청할 데이터
params = {
    'numOfRows': 1,  # 페이지당 결과 수
    'pageNo': 1,  # 페이지 번호
    'resultType': 'json',  # 결과 형식 (xml 또는 json)
    'crno': '1746110000741',  # 법인등록번호
    'bizYear': '2019',  # 사업연도
    'serviceKey': service_key  # 인증키
}

# GET 요청 보내기
response = requests.get(url, params=params)

# 응답 데이터 파싱
if response.status_code == 200:  # 성공적으로 응답 받은 경우
    result = response.json()  # json 형식으로 파싱
    print(result)
else:
    print(f'Error {response.status_code}')

JSONDecodeError: ignored

In [ ]:
import requests

url = "http://apis.data.go.kr/1160100/service/GetFinaStatInfoService_V2/getSummFinaStat_V2"
params = {
    "serviceKey": "3phTHgOnMKftn3MRUX9XsCGgzB5/VlozxJc16E68cE6ANiIoXsL9dprcwxB3c3KUfMJqaY4oPQwE8HraWBJYGg==",
    "numOfRows": 10,
    "pageNo": 1,
    "fnccmpNm": "삼성전자",
}

response = requests.get(url, params=params)

if response.status_code == 200:
    data = response.json()
    # 파싱 코드 작성
else:
    print("Error:", response.status_code)

JSONDecodeError: ignored

In [ ]:
import requests
from xml.etree import ElementTree

url = "http://apis.data.go.kr/1160100/service/GetFinaStatInfoService_V2/getSummFinaStat_V2"
params = {
    "serviceKey": "3phTHgOnMKftn3MRUX9XsCGgzB5/VlozxJc16E68cE6ANiIoXsL9dprcwxB3c3KUfMJqaY4oPQwE8HraWBJYGg==",
    "numOfRows": 10,
    "pageNo": 1,
    "fnccmpNm": "삼성전자",
}

response = requests.get(url, params=params)

if response.status_code == 200:
    root = ElementTree.fromstring(response.content)
    for item in root.findall(".//item"):
        print(item.find("basDt").text, item.find("fnclDcdNm").text, item.find("enpSaleAmt").text)
else:
    print("Error:", response.status_code)

19990212 NA 80365000000
19991210 NA 80365000000
19990511 NA 811018000000
19990608 NA 811018000000
19990720 NA 811018000000
19990812 NA 811018000000
19990921 NA 811018000000
19991015 NA 811018000000
19991110 NA 811018000000
19991222 NA 811018000000


In [ ]:
from multiprocessing import Pool
import pandas as pd
import json
import urllib.request
#한국거래소에서 종목 코드 받아옵니다. [0]은 헤더를 첫번째 행으로 지정하기 위해 사용 했습니다.
code_df = pd.read_html('http://kind.krx.co.kr/corpgeneral/corpList.do?method=download&searchType=13')[0]
#종목코드가 6자리이기 때문에 6자리를 맞춰주기 위해 설정해줍니다.
code_df.종목코드 = code_df.종목코드.map('{:06d}'.format)
#우리에게 필요한 회사명과 종목코드만 남깁니다.
code_df = code_df[['회사명', '종목코드']]
#리스트 형식으로 변경하면 네이버증권API 사용을 위한 준비가 끝납니다.
stock_list = code_df.values.tolist()

#multiprocessing을 위해 함수로 작성합니다.
def func(stock_set):
    #회사명을 가져옵니다.
    item_name = stock_set[0]
    #종목코드를 가져옵니다.
    item_code = stock_set[1]
    #네이버증권API인 요약 정보 기능을 사용 합니다.
    url = "https://api.finance.naver.com/service/itemSummary.nhn?itemcode=%s"%(item_code)
    #urllib.request를 통해 링크의 결과를 가져옵니다.
    raw_data = urllib.request.urlopen(url).read()
    #추후, 데이터 가공을 위해 json 형식으로 변경 합니다.
    json_data = json.loads(raw_data)
    #PER과 PBR을 변수로 지정합니다.
    per_value = json_data['per']
    pbr_value = json_data['pbr']
    #우선, PER 값이 있는 종목을 골라 냅니다.
    if per_value :
        #PER이 0보다 크고 2보다 작은 종목이면서 PBR이 0.7보다 작은 종목을 찾습니다.
        if 0 < per_value < 2 and 0.7 > pbr_value :
            print("[%s]의 per은 %s 입니다. (pbr %s)" %(item_name, per_value, pbr_value))

#func 함수를 stock_list 범위로 병렬 처리 합니다.
if __name__ == '__main__':
    with Pool(6) as p:
        p.map(func, stock_list)

[GS]의 per은 1.76 입니다. (pbr 0.3)
[엑세스바이오]의 per은 1.08 입니다. (pbr 0.61)
[흥국화재]의 per은 1.41 입니다. (pbr 0.29)
[유성티엔에스]의 per은 1.5 입니다. (pbr 0.19)
[SK디스커버리]의 per은 1.51 입니다. (pbr 0.22)
[티케이케미칼]의 per은 0.92 입니다. (pbr 0.27)
[HMM]의 per은 0.96 입니다. (pbr 0.47)
[SG&G]의 per은 1.4 입니다. (pbr 0.18)
[SGC이테크건설]의 per은 1.59 입니다. (pbr 0.32)
[다산네트웍스]의 per은 0.71 입니다. (pbr 0.36)
[파인테크닉스]의 per은 0.37 입니다. (pbr 0.47)
[영원무역홀딩스]의 per은 1.94 입니다. (pbr 0.35)
[한신공영]의 per은 1.94 입니다. (pbr 0.12)
[KG스틸]의 per은 1.98 입니다. (pbr 0.63)
[팜젠사이언스]의 per은 1.45 입니다. (pbr 0.5)
[휴스틸]의 per은 1.27 입니다. (pbr 0.41)
[한화]의 per은 1.91 입니다. (pbr 0.47)
[원익]의 per은 1.96 입니다. (pbr 0.3)
[이지스레지던스리츠]의 per은 1.72 입니다. (pbr 0.45)
[전방]의 per은 1.0 입니다. (pbr 0.22)
[티웨이홀딩스]의 per은 1.26 입니다. (pbr 0.41)
[한국가스공사]의 per은 1.64 입니다. (pbr 0.22)
[그래디언트]의 per은 0.73 입니다. (pbr 0.38)
[대창스틸]의 per은 1.06 입니다. (pbr 0.38)


In [ ]:
import pandas as pd
import json
import requests
from concurrent.futures import ThreadPoolExecutor

# 한국거래소에서 종목 코드 받아옵니다. [0]은 헤더를 첫번째 행으로 지정하기 위해 사용 했습니다.
code_df = pd.read_html('http://kind.krx.co.kr/corpgeneral/corpList.do?method=download&searchType=13')[0]
# 종목코드가 6자리이기 때문에 6자리를 맞춰주기 위해 설정해줍니다.
code_df.종목코드 = code_df.종목코드.map('{:06d}'.format)
# 우리에게 필요한 회사명과 종목코드만 남깁니다.
code_df = code_df[['회사명', '종목코드']]
# 리스트 형식으로 변경하면 네이버증권API 사용을 위한 준비가 끝납니다.
stock_list = code_df.values.tolist()

# multiprocessing을 위해 함수로 작성합니다.
def func(stock_set):
    # 회사명을 가져옵니다.
    item_name = stock_set[0]
    # 종목코드를 가져옵니다.
    item_code = stock_set[1]
    # 네이버증권API인 요약 정보 기능을 사용 합니다.
    url = f"https://api.finance.naver.com/service/itemSummary.nhn?itemcode={item_code}"
    # requests 패키지를 사용하여 요청을 보냅니다.
    response = requests.get(url)
    # 추후, 데이터 가공을 위해 json 형식으로 변경 합니다.
    json_data = response.json()
    # PER과 PBR을 변수로 지정합니다.
    per_value = json_data['per']
    pbr_value = json_data['pbr']
    # 우선, PER 값이 있는 종목을 골라 냅니다.
    if per_value:
        # PER이 0보다 크고 2보다 작은 종목이면서 PBR이 0.7보다 작은 종목을 찾습니다.
        if 0 < per_value < 2 and 0.7 > pbr_value:
            return f"[{item_name}]의 per은 {per_value} 입니다. (pbr {pbr_value})"
    return None

# func 함수를 stock_list 범위로 병렬 처리 합니다.
if __name__ == '__main__':
    with ThreadPoolExecutor(max_workers=6) as executor:
        results = list(executor.map(func, stock_list))

    # 결과를 출력합니다.
    for result in results:
        if result:
            print(result)

[GS]의 per은 1.76 입니다. (pbr 0.3)
[엑세스바이오]의 per은 1.08 입니다. (pbr 0.61)
[유성티엔에스]의 per은 1.5 입니다. (pbr 0.19)
[티케이케미칼]의 per은 0.92 입니다. (pbr 0.27)
[SK디스커버리]의 per은 1.51 입니다. (pbr 0.22)
[흥국화재]의 per은 1.42 입니다. (pbr 0.29)
[SG&G]의 per은 1.4 입니다. (pbr 0.18)
[SGC이테크건설]의 per은 1.59 입니다. (pbr 0.32)
[다산네트웍스]의 per은 0.71 입니다. (pbr 0.36)
[HMM]의 per은 0.96 입니다. (pbr 0.47)
[영원무역홀딩스]의 per은 1.94 입니다. (pbr 0.35)
[한신공영]의 per은 1.94 입니다. (pbr 0.12)
[파인테크닉스]의 per은 0.38 입니다. (pbr 0.48)
[KG스틸]의 per은 1.99 입니다. (pbr 0.63)
[한화]의 per은 1.91 입니다. (pbr 0.47)
[팜젠사이언스]의 per은 1.46 입니다. (pbr 0.5)
[휴스틸]의 per은 1.27 입니다. (pbr 0.41)
[티웨이홀딩스]의 per은 1.26 입니다. (pbr 0.41)
[그래디언트]의 per은 0.73 입니다. (pbr 0.38)
[대창스틸]의 per은 1.06 입니다. (pbr 0.38)
[이지스레지던스리츠]의 per은 1.72 입니다. (pbr 0.45)
[전방]의 per은 1.0 입니다. (pbr 0.22)
[한국가스공사]의 per은 1.64 입니다. (pbr 0.22)
[원익]의 per은 1.96 입니다. (pbr 0.3)


In [ ]:
import pandas as pd
import json
import urllib.request
from concurrent.futures import ProcessPoolExecutor

# 한국거래소에서 종목 코드 받아옵니다. [0]은 헤더를 첫번째 행으로 지정하기 위해 사용 했습니다.
code_df = pd.read_html('http://kind.krx.co.kr/corpgeneral/corpList.do?method=download&searchType=13')[0]
# 종목코드가 6자리이기 때문에 6자리를 맞춰주기 위해 설정해줍니다.
code_df.종목코드 = code_df.종목코드.map('{:06d}'.format)
# 우리에게 필요한 회사명과 종목코드만 남깁니다.
code_df = code_df[['회사명', '종목코드']]
# 리스트 형식으로 변경하면 네이버증권API 사용을 위한 준비가 끝납니다.
stock_list = code_df.values.tolist()

# multiprocessing을 위해 함수로 작성합니다.
def func(stock_set):
    # 회사명을 가져옵니다.
    item_name = stock_set[0]
    # 종목코드를 가져옵니다.
    item_code = stock_set[1]
    # 네이버증권API인 요약 정보 기능을 사용 합니다.
    url = "https://api.finance.naver.com/service/itemSummary.nhn?itemcode=%s"%(item_code)
    # urllib.request를 통해 링크의 결과를 가져옵니다.
    raw_data = urllib.request.urlopen(url).read()
    # 추후, 데이터 가공을 위해 json 형식으로 변경 합니다.
    json_data = json.loads(raw_data)
    # PER과 PBR을 변수로 지정합니다.
    per_value = json_data['per']
    pbr_value = json_data['pbr']
    # 우선, PER 값이 있는 종목을 골라 냅니다.
    if per_value:
        # PER이 0보다 크고 2보다 작은 종목이면서 PBR이 0.7보다 작은 종목을 찾습니다.
        if 0 < per_value < 2 and 0.7 > pbr_value:
            print("[%s]의 per은 %s 입니다. (pbr %s)" %(item_name, per_value, pbr_value))

# func 함수를 stock_list 범위로 병렬 처리 합니다.
if __name__ == '__main__':
    with ProcessPoolExecutor() as executor:
        executor.map(func, stock_list)

[GS]의 per은 1.76 입니다. (pbr 0.3)
[엑세스바이오]의 per은 1.08 입니다. (pbr 0.61)
[유성티엔에스]의 per은 1.5 입니다. (pbr 0.19)
[티케이케미칼]의 per은 0.91 입니다. (pbr 0.27)
[SK디스커버리]의 per은 1.51 입니다. (pbr 0.22)
[흥국화재]의 per은 1.41 입니다. (pbr 0.29)
[SG&G]의 per은 1.4 입니다. (pbr 0.18)
[SGC이테크건설]의 per은 1.59 입니다. (pbr 0.32)
[다산네트웍스]의 per은 0.71 입니다. (pbr 0.36)
[HMM]의 per은 0.96 입니다. (pbr 0.47)
[영원무역홀딩스]의 per은 1.94 입니다. (pbr 0.35)
[한신공영]의 per은 1.93 입니다. (pbr 0.12)
[파인테크닉스]의 per은 0.38 입니다. (pbr 0.48)
[KG스틸]의 per은 1.99 입니다. (pbr 0.63)
[한화]의 per은 1.91 입니다. (pbr 0.47)
[팜젠사이언스]의 per은 1.45 입니다. (pbr 0.5)
[휴스틸]의 per은 1.27 입니다. (pbr 0.41)
[티웨이홀딩스]의 per은 1.25 입니다. (pbr 0.4)
[그래디언트]의 per은 0.73 입니다. (pbr 0.38)
[대창스틸]의 per은 1.05 입니다. (pbr 0.38)
[이지스레지던스리츠]의 per은 1.72 입니다. (pbr 0.45)
[전방]의 per은 0.99 입니다. (pbr 0.22)
[한국가스공사]의 per은 1.64 입니다. (pbr 0.22)
[원익]의 per은 1.96 입니다. (pbr 0.3)


In [1]:
import pandas as pd
import json
import requests
from concurrent.futures import ThreadPoolExecutor

# 한국거래소에서 종목 코드 받아옵니다. [0]은 헤더를 첫번째 행으로 지정하기 위해 사용 했습니다.
code_df = pd.read_html('http://kind.krx.co.kr/corpgeneral/corpList.do?method=download&searchType=13')[0]
# 종목코드가 6자리이기 때문에 6자리를 맞춰주기 위해 설정해줍니다.
code_df.종목코드 = code_df.종목코드.map('{:06d}'.format)
# 우리에게 필요한 회사명과 종목코드만 남깁니다.
code_df = code_df[['회사명', '종목코드']]
# 리스트 형식으로 변경하면 네이버증권API 사용을 위한 준비가 끝납니다.
stock_list = code_df.values.tolist()

# multiprocessing을 위해 함수로 작성합니다.
def func(stock_set):
    # 회사명을 가져옵니다.
    item_name = stock_set[0]
    # 종목코드를 가져옵니다.
    item_code = stock_set[1]
    # 네이버증권API인 요약 정보 기능을 사용 합니다.
    url = f"https://api.finance.naver.com/service/itemSummary.nhn?itemcode={item_code}"
    # requests 패키지를 사용하여 요청을 보냅니다.
    response = requests.get(url)
    # 추후, 데이터 가공을 위해 json 형식으로 변경 합니다.
    json_data = response.json()
    # PER과 PBR을 변수로 지정합니다.
    per_value = json_data['per']
    pbr_value = json_data['pbr']
    # 우선, PER 값이 있는 종목을 골라 냅니다.
    if per_value:
        # PER이 0보다 크고 2보다 작은 종목이면서 PBR이 0.7보다 작은 종목을 찾습니다.
        if 0 < per_value < 2 and 0.7 > pbr_value:
            return f"[{item_name}]의 per은 {per_value} 입니다. (pbr {pbr_value})"
    return None

# func 함수를 stock_list 범위로 병렬 처리 합니다.
if __name__ == '__main__':
    with ThreadPoolExecutor(max_workers=6) as executor:
        results = list(executor.map(func, stock_list))

# 문자열과 None 값을 포함하는 리스트에서 None 값 제거
results = list(filter(None, results))

# 결과를 DataFrame으로 변환
results_df = pd.DataFrame([result[1:-1].split(']의 per은 ') for result in results], columns=['회사명', 'per_pbr'])
results_df['회사명'] = results_df['회사명'].str.strip()

file_path = "국민연금공단_국내주식 투자정보_20211231.csv"
df = pd.read_csv(file_path, encoding='euc-kr')

# "종목명" 컬럼이 results_df의 "회사명" 컬럼과 일치하는 행 필터링
intersection = df['종목명'].isin(results_df['회사명'])
matched_df = df[intersection]

# 결과 출력
pd.set_option('display.width', 1000)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.max_columns', 10)
pd.set_option('display.precision', 2)

print("국민연금이 보유한 주식 중 저평가된 주식 리스트\n")
print(matched_df.to_string(index=False))

국민연금이 보유한 주식 중 저평가된 주식 리스트

  번호       종목명  평가액(억 원)  자산군 내 비중(퍼센트)  지분율(퍼센트)
  49       HMM      5923           0.62      4.50
  85    한국가스공사      3235           0.17      8.96
 100        GS      2617           0.17      7.20
 137        한화      1765           0.11      7.50
 257   SK디스커버리       541           0.04      6.13
 339   영원무역홀딩스       237           0.03      3.72
 401    파인테크닉스       157           0.00      2.55
 540      한신공영        58           0.01      3.19
 692    티케이케미칼        27           0.00      0.53
 919      흥국화재         7           0.01      0.29
 939    다산네트웍스         6           0.00      0.22
1028    팜젠사이언스         3           0.01      0.17
1147 이지스레지던스리츠         1           0.01      0.06
1163    티웨이홀딩스         1           0.01      0.07
1165       휴스틸         0           0.01      0.04
